In [1]:
import os
import json
from typing import List, Dict, Optional
import time
from datetime import datetime
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset

class PromptEngineeringTrainer:
    def __init__(self, model_name: str = "distilbert-base-uncased", num_labels: int = 3):
        """
        Initialize the trainer with model settings
        
        Args:
            model_name: Name of the model to use
            num_labels: Number of labels for classification task
        """
        self.model_name = model_name
        self.tokenizer = DistilBertTokenizer.from_pretrained(model_name)
        self.model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
        self.results = []
        
    def tokenize_data(self, data: List[Dict[str, str]]) -> Dataset:
        """
        Tokenize the data into input format compatible with the model
        
        Args:
            data: List of dictionaries containing input/output pairs
            
        Returns:
            Tokenized Dataset
        """
        texts = [item['input'] for item in data]
        labels = [self.get_label(item['output']) for item in data]  # Map output to numerical labels
        encodings = self.tokenizer(texts, truncation=True, padding=True, max_length=128)
        return Dataset.from_dict({'input_ids': encodings['input_ids'], 'attention_mask': encodings['attention_mask'], 'labels': labels})

    def get_label(self, label: str) -> int:
        """
        Convert the output label to a numerical representation
        
        Args:
            label: The label to convert ('positive', 'negative', 'neutral')
            
        Returns:
            Integer representation of the label
        """
        label_map = {'positive': 0, 'negative': 1, 'neutral': 2}
        return label_map.get(label.lower(), -1)  # Default to -1 if label is unknown
    
    def train_and_evaluate(self, training_data: List[Dict[str, str]], test_size: float = 0.2):
        """
        Train and evaluate the model
        
        Args:
            training_data: List of dictionaries containing input/output pairs
            test_size: Proportion of data to use for testing
        """
        # Split data into train and test sets
        train_data, test_data = train_test_split(training_data, test_size=test_size, random_state=42)
        
        # Tokenize the data
        train_dataset = self.tokenize_data(train_data)
        test_dataset = self.tokenize_data(test_data)
        
        # Training arguments
        training_args = TrainingArguments(
            output_dir="./results",
            evaluation_strategy="epoch",
            per_device_train_batch_size=8,
            per_device_eval_batch_size=8,
            num_train_epochs=3,
            logging_dir="./logs",
        )
        
        # Trainer
        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=test_dataset,
        )
        
        # Train the model
        trainer.train()
        
        # Evaluate the model
        eval_results = trainer.evaluate()
        accuracy = eval_results['eval_accuracy'] if 'eval_accuracy' in eval_results else 0
        
        return accuracy
    
    def save_results(self, filename: str):
        """
        Save training results to a JSON file
        
        Args:
            filename: Name of the file to save results
        """
        output = {
            'model': self.model_name,
            'timestamp': datetime.now().isoformat(),
            'results': self.results
        }
        
        with open(filename, 'w') as f:
            json.dump(output, f, indent=2)

# Example usage
def main():
    # Sample training data (sentiment analysis example)
    training_data = [
        {'input': 'This movie was absolutely fantastic!', 'output': 'positive'},
        {'input': 'I really hated every minute of it.', 'output': 'negative'},
        {'input': 'The product arrived on time but was damaged.', 'output': 'negative'},
        {'input': 'Great customer service and fast shipping!', 'output': 'positive'},
        {'input': 'It was okay, nothing special.', 'output': 'neutral'},
        # Add more examples as needed
    ]
    
    # Initialize trainer
    trainer = PromptEngineeringTrainer(
        model_name="distilbert-base-uncased",  # You can choose other models like 'bert-base-uncased', 't5-small', etc.
        num_labels=3  # Number of sentiment classes (positive, negative, neutral)
    )
    
    # Train and evaluate
    accuracy = trainer.train_and_evaluate(training_data)
    print(f"Model accuracy: {accuracy:.2%}")
    
    # Save results
    trainer.save_results("training_results.json")

if __name__ == "__main__":
    main()


c:\Users\naran\anaconda3\envs\finetune_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\naran\anaconda3\envs\finetune_env\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
                                             
 33%|███▎      | 1/3 [00:00<00:00,  2.30it/s]

{'eval_loss': 1.0830050706863403, 'eval_runtime': 0.0316, 'eval_samples_per_second': 31.664, 'eval_steps_per_second': 31.664, 'epoch': 1.0}


                                             
100%|██████████| 3/3 [00:00<00:00,  5.99it/s]

{'eval_loss': 1.1082273721694946, 'eval_runtime': 0.0377, 'eval_samples_per_second': 26.524, 'eval_steps_per_second': 26.524, 'epoch': 2.0}


                                             
100%|██████████| 3/3 [00:08<00:00,  2.73s/it]


{'eval_loss': 1.1225230693817139, 'eval_runtime': 0.0134, 'eval_samples_per_second': 74.783, 'eval_steps_per_second': 74.783, 'epoch': 3.0}
{'train_runtime': 8.1853, 'train_samples_per_second': 1.466, 'train_steps_per_second': 0.367, 'train_loss': 1.0428200562795003, 'epoch': 3.0}


100%|██████████| 1/1 [00:00<00:00, 1002.46it/s]

Model accuracy: 0.00%
